# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pystan
  error: subprocess-exited-with-error
  
  × Running setup.py install for pystan did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Running setup.py install for pystan ... error
error: legacy-install-failure

× Encountered error while trying to install package.
╰─> pystan

note: This is an issue with the package mentioned above, not pip.
hint: See above for output from the failure.
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
 

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
#from fbprophet import Prophet
#from fbprophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-05-04' 
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2675,2023-04-29,"$29,336.57","$29,452.46","$29,088.04","$29,248.49","$29,248.49",10662634333
2676,2023-04-30,"$29,245.52","$29,952.03","$29,114.02","$29,268.81","$29,268.81",14652199272
2677,2023-05-01,"$29,227.10","$29,329.94","$27,680.79","$28,091.57","$28,091.57",18655599976
2678,2023-05-02,"$28,087.18","$28,881.30","$27,924.12","$28,680.54","$28,680.54",16432924527
2679,2023-05-03,"$28,680.49","$29,259.53","$28,178.39","$29,006.31","$29,006.31",19122972518


In [ ]:
# Validamos que no hay datos vacíos
btc_df.isnull

<bound method DataFrame.isnull of            Date       Open       High        Low      Close  Adj Close  \
0    2016-01-01    $430.72    $436.25    $427.52    $434.33    $434.33   
1    2016-01-02    $434.62    $436.06    $431.87    $433.44    $433.44   
2    2016-01-03    $433.58    $433.74    $424.71    $430.01    $430.01   
3    2016-01-04    $430.06    $434.52    $429.08    $433.09    $433.09   
4    2016-01-05    $433.07    $434.18    $429.68    $431.96    $431.96   
...         ...        ...        ...        ...        ...        ...   
2675 2023-04-29 $29,336.57 $29,452.46 $29,088.04 $29,248.49 $29,248.49   
2676 2023-04-30 $29,245.52 $29,952.03 $29,114.02 $29,268.81 $29,268.81   
2677 2023-05-01 $29,227.10 $29,329.94 $27,680.79 $28,091.57 $28,091.57   
2678 2023-05-02 $28,087.18 $28,881.30 $27,924.12 $28,680.54 $28,680.54   
2679 2023-05-03 $28,680.49 $29,259.53 $28,178.39 $29,006.31 $29,006.31   

           Volume  
0        36278900  
1        30096600  
2        39633800

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df=btc_df[['Date','Open']] 


# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df =df.rename(columns=new_names) 

In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicia= ('BTC-USD')
#from fbprophet import Prophet

m = Prophet(seasonality_mode='multiplicative')


In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
3040,2024-04-28
3041,2024-04-29
3042,2024-04-30
3043,2024-05-01
3044,2024-05-02


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future
future = future[ future['ds'].dt.dayofweek < 5 ]

<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)
from prophet.plot import plot_plotly,plot_components_plotly

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2170,2024-04-26,"$-10,438.42","$-30,812.27","$9,081.67"
2171,2024-04-29,"$-10,495.44","$-29,451.83","$7,869.96"
2172,2024-04-30,"$-10,493.78","$-29,650.84","$7,371.02"
2173,2024-05-01,"$-10,470.33","$-29,392.32","$8,110.28"
2174,2024-05-02,"$-10,462.69","$-29,370.06","$8,488.45"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)